<a href="https://colab.research.google.com/github/akshithamaddi/Akshitha_INFO5731_Spring2020/blob/master/Copy_of_Copy_of_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [79]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [80]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1,151 kB]
Get:15

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [0]:
#scraping the data from web to obtain the file
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
url = "https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(url)
time.sleep(10)
Title = []
date = []
content= []
page=0
while True:
    time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all("div", {"class": "review-container"})
    if(page==3) :
      for result in results:
          Title.append(result.find("a", {"class": "title"}).get_text().strip())
          date.append(result.find("span", {"review-date"}).get_text().strip())
          content.append(result.find("div", {"class": "content"}).get_text().strip())
    if len(driver.find_elements_by_css_selector('.load-more-data')) > 0:
          driver.find_element_by_css_selector('.load-more-data').click()
          page+=1
          if int(page)>3:
           break
    else:
          break
df = pd.DataFrame({"Title": Title, "date": date, "content": content})
df.to_csv("output_review.csv")

In [0]:
#Data Cleaning and Processing
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from textblob import Word

ps = PorterStemmer()
lemma = WordNetLemmatizer()

pd.read_csv('output_review.csv', encoding='utf-8')
df = pd.read_csv('output_review.csv')

df.columns = ['','Title', 'date', 'content']
#lowercase and removing special characters and punctuation marks
df['Title'] = df['Title'].str.lower()
df['content'] = df['content'].str.lower()
df['Title']= df['Title'].str.replace(r'[^\w\s]+','')
df['content']= df['content'].str.replace(r'[^\w\s]+','')
#removing numbers
df['Title'] = df['Title'].str.replace('\d+', '')
df['content'] = df['content'].str.replace('\d+', '')
#removing stopwords
stop = stopwords.words('english')
df['Title'] = df['Title'].apply(lambda x: [item for item in str.split(x) if item not in stop])
df['content'] = df['content'].apply(lambda x: [item for item in str.split(x) if item not in stop])
#lemmatization
df['Title']= df['Title'].apply(lambda x : [lemma.lemmatize(y) for y in x])
#df['Title'] = df['Title'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))
df['content']= df['content'].apply(lambda x : [lemma.lemmatize(y) for y in x])
#stemming
df['Title']= df['Title'].apply(lambda x : [ps.stem(y) for y in x])
df['content']= df['content'].apply(lambda x : [ps.stem(y) for y in x])

def remove_punc(text):
  no_punc=" ".join([c for c in text if c not in string.punctuation])
  return no_punc
df['Title']= df['Title'].apply(lambda x : remove_punc(x))
df['content']= df['content'].apply(lambda x : remove_punc(x))

#Adding columns for clean_content and clean_title in a csv
csv_input = pd.read_csv('output_review.csv')
csv_input['clean_title'] = df['Title']
csv_input['clean_content'] = df['content']
csv_input.to_csv('output_review.csv', index=False)

In [83]:
#frequency of 3-grams
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
data = pd.read_csv('output_review.csv', low_memory=False)
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(data['clean_title'].values.astype('U')) 
frequencies = sum(sparse_matrix).toarray()[0]
df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['Frequency'])
df

,Frequency
absolut masterpiec phoenix,1
account rate disappoint,1
account tell everyon,1
act perform ive,1
act perform phoenix,1
...,...
worthi act perform,1
would call masterpiec,1
write review that,1
year best actor,1


In [0]:
#collecting the required column from a csv file and saving it in text file
#file is used to calculate the probabilities of bi-gram
import csv
with open('output_review.csv') as f:
 reader = csv.reader(f)
 next(reader, None)
 clean_title = [row[4] for row in reader]
with open('clean_title.txt', mode="w") as outfile:
    for s in clean_title:
        outfile.write("%s\n" % s)

In [0]:
#Calculating probabilities of bi-grams using a function
def bigramEstimation(file):
    lst = []
    unigrams = {}
    bigrams = {} 
    text = open(file, 'r').read()
    lst = text.strip().split()
    del text 
    for l in lst:
        if not l in unigrams:
            unigrams[l] = 1
        else:
            unigrams[l] += 1
    for i in range(len(lst) - 1):
        temp = (lst[i], lst[i+1])
        if not temp in bigrams:
            bigrams[temp] = 1
        else:
          bigrams[temp] += 1
    print('Generated ', len(bigrams), ' bigrams')
    total_corpus = sum(unigrams.values())
    for k,v in bigrams.items():
        first_word = k[0]
        first_word_count = unigrams[first_word]
        bi_prob = bigrams[k] / unigrams[first_word]
        if(v == 2):
         print(k[0],k[1],v,bi_prob)        

In [87]:
bigramEstimation('clean_title.txt')

Generated  341  bigrams
amaz movi 2 1.0
joker best 2 0.14285714285714285
one best 2 1.0
act perform 2 0.6666666666666666
ive ever 2 1.0
extrem overr 2 1.0
best comic 2 0.16666666666666666
comic book 2 1.0
perform phoenix 2 0.5
best movi 2 0.16666666666666666
dont get 2 0.2857142857142857
real joker 2 0.4
best dc 2 0.16666666666666666
sinc dark 2 1.0
movi year 2 0.11764705882352941
masterpiec joker 2 0.2


In [88]:
!pip install spacy
!python -m spacy download en_core_web_sm


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [89]:
#extracting nouns and calculating the noun frequencies 
import spacy
import csv
from tabulate import tabulate


nlp = spacy.load("en")
file = open("clean_title.txt", "r")
doc = nlp(file.read())
noun_phrases = []
for np in doc.noun_chunks:
  noun_phrases.append(np.text)
print(noun_phrases)
dfn = pd.DataFrame(noun_phrases, columns = ['noun_phrases'])
word_vectorizer = CountVectorizer(ngram_range=(2,7), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(dfn['noun_phrases'].values.astype('U')) 
frequencies = sum(sparse_matrix).toarray()[0]
dff = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['Frequency'])
dff['Noun_Probabilities'] = dff['Frequency'] / dff['Frequency'].max()
result = dff.astype(object).transpose() 
print(tabulate(result, headers='keys', tablefmt='grid'))

['viewer actual', 'tiff', 'wit film', 'believ hype absolut masterpiec', 'phoenix', 'legend', 'outstand movi haunt', 'best charact', 'certain peopl relat\nperfect everi aspect', 'masterpiec', 'amaz movi', 'psycholog studi', 'rather superhero flick', 'joaquin oscar joker', 'best dark suspens thriller darker dark knight', 'venic review', 'final real movi', 'good lord', 'spoon feed cgi fuel faux drama', 'real life', 'joker\noscar', 'phoenix', 'critic', 'joker endgam', 'masterpiec', 'one best act', 'i', 'smile', 'overr', 'believ hype', 'masterpiec', 'that life', 'extrem overr', 'broken man', 'joker', 'brilliant best joker', 'i', 'movi', 'old peopl', 'clown princ crime arriv', 'probabl', 'one best comic book movi', 'phoenix', 'worthi act', 'phoenix', 'watch', 'joker', 'enough becom joker', 'we', 'account', 'everyon good', 'miser unpleas slog movi noth', 'stun\nanyon rate movi poorli clearli', 'cinema', 'yike', 'bewar anyon hail masterpiec', 'astonish masterpiec', 'nonsens plot', 'everyon bra

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [90]:
#calculating the term-frequency
df = pd.read_csv("output_review.csv")
tf2 = df.dropna()
tf1 = (tf2['clean_title'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index())
tf1.columns = ['words','tf']
tf1

,words,tf
0,actual,1.0
1,legend,1.0
2,film,6.0
3,didnt,2.0
4,wit,1.0
...,...,...
203,far,1.0
204,date,1.0
205,deliv,1.0
206,stori,1.0


In [91]:
#building the document term weights(tf*idf)
import numpy as np
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(df.shape[0]/(len(tf2[tf2['clean_title'].str.contains(word)])))
tf1['tf*idf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tf*idf
0,actual,1.0,4.605170,4.605170
1,legend,1.0,4.605170,4.605170
2,film,6.0,2.813411,16.880464
3,didnt,2.0,3.912023,7.824046
4,wit,1.0,4.605170,4.605170
...,...,...,...,...
203,far,1.0,4.605170,4.605170
204,date,1.0,4.605170,4.605170
205,deliv,1.0,4.605170,4.605170
206,stori,1.0,4.605170,4.605170


In [0]:
#Cosine similarity for all the documents
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
doc_set = tf2['clean_title'].values.tolist()
query_set = "As a viewer that actually went to TIFF and witnessed this film and didn't want to believe the hype, it is an absolute MASTERPIECE and Phoenix is a certified legend."
query_set = [query_set]
stopWords = stopwords.words('english')
vectorizer = CountVectorizer(stop_words = stopWords)
transformer = TfidfTransformer()
docVectorizerArray = vectorizer.fit_transform(doc_set).toarray()
queryVectorizerArray = vectorizer.transform(query_set).toarray()
cx = lambda a, b : np.inner(a, b)/(LA.norm(a)*LA.norm(b))
result = []
for vector in docVectorizerArray:
        for testV in queryVectorizerArray:
            cosine = cx(vector, testV)
            result.append(cosine)           

In [94]:
#ranking the dicuments based on  cosine similarity
new = tf2.filter(['Unnamed','clean_title'], axis=1)
se = pd.Series(result)
new['Cosine_similarity'] = se.values
new.drop(new.loc[new['Cosine_similarity']==0].index, inplace=True)
new["Rank"] = new["Cosine_similarity"].rank().astype(int)
new.sort_values("Cosine_similarity", inplace = True) 
new

,clean_title,Cosine_similarity,Rank
37,worthi act perform phoenix worth watch joker s...,0.088388,1
36,probabl one best comic book movi perform phoen...,0.102062,2
93,far best dc movi date joaquin phoenix perfect,0.125000,3
83,masterpiec movi year best actor joaquin phoenix,0.133631,4
88,went blind didnt enjoy realli disappoint,0.144338,5
66,overr badli direct film mislead titl,0.144338,5
94,joaquin phoenix deliv stori doesnt,0.158114,7
48,made account rate disappoint film,0.158114,7
7,went second time watch,0.176777,10
80,great act terribl film,0.176777,10


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
https://github.com/akshithamaddi/Akshitha_INFO5731_Spring2020/blob/master/Assignment5_File_5731.csv